<a href="https://colab.research.google.com/github/phdosilva/taes/blob/main/TAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação da biblioteca do Z3 solver

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 25.4 MB/s eta 0:00:00


# Função que busca conflitos nos requisitos convertidos para Lógica de 1ª ordem

Para eficiência computacional, serão checados subconjuntos de, no máximo, 3 requisitos.

In [2]:
from z3 import *
from itertools import combinations

def find_unsat_subsets(constraints):
    solver = Solver()
    solver.add(constraints)

    if solver.check() != unsat:
        print("The full set of constraints is satisfiable.")
        return

    print("The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...")

    unsat_subsets = []
    max_size = min(3, len(constraints))

    for i in range(1, max_size + 1):
        for subset in combinations(constraints, i):
            solver = Solver()
            solver.add(subset)
            if solver.check() == unsat:
                unsat_subsets.append(set(subset))

    for idx, subset in enumerate(unsat_subsets, 1):
        print(f"UNSAT subset {idx}:")
        for c in subset:
            print(c)
        print("---")

    return unsat_subsets

# Funcionamento

Para cada LLM, foi introduzido o seguinte prompt:

```
You are an virtual assistant, especialized in converting software requirements - which are expressed in natural language form - into first-order logic form.

Your task is to:
1. Read the entry, where each line is a requirement; and
2. Convert each requirement into its logic form;
3. Convert each logic form into z3-solver (python library) code.

Your returned message shall only contain the z3-solver code, without other explanations or process description.

The z3 constraints should be in a array named "constraints", so we only use the "s.add()" method once, to add all the constraints

Key points to consider while performing the task:
- Take into account other requirements to unify logic symbols;
- The requirements entry does not need addition of new requirements;
- Do not provide analysis on requirement contradiction. Your task is only to make the conversions as described above.

Entry:
"""
<Requisitos separados por uma nova linha>
"""
```

Após a obtenção do código, realizou-se:

1. Verificação de satisfatibilidade; e
1. Obtenção dos subconjuntos confiltantes, caso o conjunto tenha sido insatisfatível

# Dataset Broker

Requirements:
```
After 3 failed login attempts, the system must not allow account access.
When locking an account, the system must send a notification email.
When locking an account, the system must send an SMS unless email was sent.
The system must invoice brokers annually using CDR.
The system must invoice brokers monthly using CDR.
The system must not allow invoice deletion.
Brokers must be allowed to delete subscriber accounts and data.
Invoices must only be stored in subscriber accounts.
PBX accounts must not have virtual circuits.
The system must deactivate unpaid PBX virtual circuits.
Brokers must be allowed to access the rate table.
Only administrators must access and manage rate table permissions.
The system must assign unique user IDs.
Users must be allowed to customize ID prefixes.
The system must send invoice/approval/payment notifications to brokers and subscribers.
The system must integrate with payment gateways.
Payment history must be recorded and accessible.
The system must log all user actions (deletions/modifications/accesses).
The system must provide reporting/analytics for admins and brokers.
The system must support multi-currency invoicing/payments when required.
Monthly backups and recovery plans must be maintained.
CRM/ERP integration must be available when required.
Brokers must be allowed to bulk-generate invoices.
Accounting software integration must be available.
Brokers must have dashboards showing account status/earnings.
The system must calculate broker commissions per paid invoice.
Brokers must be allowed to set subscriber credit limits.
Admins must be allowed to terminate accounts for violations.
Admins must be allowed to temporarily suspend accounts.
New broker registrations must require credential verification.
Brokers must not modify closed account data.
Monthly account audits must be performed.
Weekly account audits must be performed.
Foreign account creation must not be allowed.
Inactive foreign accounts (>3 months) must be locked.
Brokers must be allowed to print rate tables.
Brokers must be allowed to modify new subscription accounts.
Brokers must be allowed to assign custom numbers to new accounts.
Overdue invoices must be highlighted in dashboards.
Brokers must be allowed to delete subscriber accounts/invoices.
Overdue invoice highlighting must be configurable.
The system must auto-generate unique subscription numbers.
Brokers must be allowed to modify closed account data.
Invoices >60 days old must be flagged as overdue.
Flagged invoices must enter cleanup scheduling.
Overdue invoices must not be deleted.
Admins must be allowed to view closed broker accounts.
Users with closed-account view access must be allowed to reopen them.
Admins must not be allowed to reopen closed accounts.
Payment Gateway integration must record payment history.
When payment history is recorded, monthly backups must occur.
Payment Gateway integration must prevent monthly invoice table backups.
Inactive foreign accounts must be locked.
Locked foreign accounts must be closed after 30 days.
Inactive foreign accounts must not be closed.
The system must not allow PBX virtual circuits.
The system must deactivate unpaid PBX circuits.
```

## ChatGPT 4

In [3]:
from z3 import *

# Define variables
failed_logins = Int('failed_logins')
account_locked = Bool('account_locked')
email_sent = Bool('email_sent')
sms_sent = Bool('sms_sent')
invoice_deleted = Bool('invoice_deleted')
broker = Bool('broker')
subscriber = Bool('subscriber')
pbx_account = Bool('pbx_account')
virtual_circuit = Bool('virtual_circuit')
pbx_unpaid = Bool('pbx_unpaid')
rate_table_access = Bool('rate_table_access')
rate_table_permissions = Bool('rate_table_permissions')
admin = Bool('admin')
user_has_id = Bool('user_has_id')
custom_prefix = Bool('custom_prefix')
integration_payment_gateway = Bool('integration_payment_gateway')
payment_history = Bool('payment_history')
logging_enabled = Bool('logging_enabled')
analytics_admins_brokers = Bool('analytics_admins_brokers')
multi_currency = Bool('multi_currency')
backup_monthly = Bool('backup_monthly')
crm_erp_integration = Bool('crm_erp_integration')
bulk_invoice = Bool('bulk_invoice')
accounting_integration = Bool('accounting_integration')
broker_dashboard = Bool('broker_dashboard')
broker_commission = Bool('broker_commission')
credit_limit = Bool('credit_limit')
account_termination = Bool('account_termination')
account_suspension = Bool('account_suspension')
broker_registration_verified = Bool('broker_registration_verified')
modify_closed_account = Bool('modify_closed_account')
account_audit_monthly = Bool('account_audit_monthly')
account_audit_weekly = Bool('account_audit_weekly')
foreign_account_creation = Bool('foreign_account_creation')
foreign_account_active = Bool('foreign_account_active')
foreign_account_locked = Bool('foreign_account_locked')
foreign_account_closed = Bool('foreign_account_closed')
rate_table_print = Bool('rate_table_print')
modify_new_subscriptions = Bool('modify_new_subscriptions')
assign_custom_numbers = Bool('assign_custom_numbers')
overdue_highlight = Bool('overdue_highlight')
delete_subscriber_data = Bool('delete_subscriber_data')
overdue_highlight_configurable = Bool('overdue_highlight_configurable')
unique_subscription_number = Bool('unique_subscription_number')
invoice_flagged = Bool('invoice_flagged')
invoice_days = Int('invoice_days')
invoice_cleanup = Bool('invoice_cleanup')
view_closed_broker_accounts = Bool('view_closed_broker_accounts')
reopen_closed_accounts = Bool('reopen_closed_accounts')
payment_backup_dependency = Bool('payment_backup_dependency')
prevent_invoice_table_backup = Bool('prevent_invoice_table_backup')

s = Solver()

# Constraints
constraints = [
    Implies(failed_logins >= 3, account_locked),
    Implies(account_locked, email_sent),
    Implies(account_locked, Or(email_sent, sms_sent)),
    Not(invoice_deleted),
    Implies(broker, delete_subscriber_data),
    Implies(subscriber, Not(delete_subscriber_data)),
    Implies(pbx_account, Not(virtual_circuit)),
    Implies(pbx_unpaid, Not(virtual_circuit)),
    Implies(broker, rate_table_access),
    Implies(rate_table_permissions, admin),
    Implies(user_has_id, custom_prefix),
    Implies(Or(broker, subscriber), integration_payment_gateway),
    Implies(integration_payment_gateway, payment_history),
    logging_enabled,
    analytics_admins_brokers,
    Implies(multi_currency, True),
    backup_monthly,
    Implies(crm_erp_integration, True),
    Implies(broker, bulk_invoice),
    Implies(accounting_integration, True),
    Implies(broker, broker_dashboard),
    Implies(broker, broker_commission),
    Implies(broker, credit_limit),
    Implies(admin, account_termination),
    Implies(admin, account_suspension),
    Implies(broker_registration_verified, True),
    Implies(broker, Not(modify_closed_account)),
    account_audit_monthly,
    account_audit_weekly,
    Not(foreign_account_creation),
    Implies(Not(foreign_account_active), foreign_account_locked),
    Implies(foreign_account_locked, Not(foreign_account_active)),
    Implies(foreign_account_locked, foreign_account_closed),
    Implies(foreign_account_active, Not(foreign_account_closed)),
    Implies(broker, rate_table_print),
    Implies(broker, modify_new_subscriptions),
    Implies(broker, assign_custom_numbers),
    Implies(invoice_days > 60, invoice_flagged),
    Implies(invoice_flagged, invoice_cleanup),
    Implies(admin, view_closed_broker_accounts),
    Implies(view_closed_broker_accounts, reopen_closed_accounts),
    Implies(admin, Not(reopen_closed_accounts)),
    Implies(integration_payment_gateway, payment_backup_dependency),
    Implies(payment_backup_dependency, backup_monthly),
    Implies(integration_payment_gateway, prevent_invoice_table_backup),
    Implies(pbx_account, Not(virtual_circuit))
]

s.add(constraints)

In [4]:
s.check()

sat

In [5]:
find_unsat_subsets(constraints)

The full set of constraints is satisfiable.


## Deepseek R1

In [6]:
from z3 import *

# Declarations
failed_login_attempts = Int('failed_login_attempts')
account_access_allowed = Bool('account_access_allowed')
account_locked = Bool('account_locked')
notification_email_sent = Bool('notification_email_sent')
sms_sent = Bool('sms_sent')
invoice_annually = Bool('invoice_annually')
invoice_monthly = Bool('invoice_monthly')
invoice_deletion_allowed = Bool('invoice_deletion_allowed')
broker_delete_subscriber_allowed = Bool('broker_delete_subscriber_allowed')
invoices_stored_in_subscriber_accounts = Bool('invoices_stored_in_subscriber_accounts')
pbx_accounts_have_virtual_circuits = Bool('pbx_accounts_have_virtual_circuits')
deactivate_unpaid_pbx_circuits = Bool('deactivate_unpaid_pbx_circuits')
broker_access_rate_table = Bool('broker_access_rate_table')
admin_manage_rate_table_permissions = Bool('admin_manage_rate_table_permissions')
unique_user_ids = Bool('unique_user_ids')
customize_id_prefixes = Bool('customize_id_prefixes')
send_notifications = Bool('send_notifications')
integrate_payment_gateways = Bool('integrate_payment_gateways')
record_payment_history = Bool('record_payment_history')
log_user_actions = Bool('log_user_actions')
reporting_analytics = Bool('reporting_analytics')
multi_currency_support = Bool('multi_currency_support')
monthly_backups = Bool('monthly_backups')
crm_erp_integration = Bool('crm_erp_integration')
bulk_generate_invoices = Bool('bulk_generate_invoices')
accounting_software_integration = Bool('accounting_software_integration')
broker_dashboards = Bool('broker_dashboards')
calculate_broker_commissions = Bool('calculate_broker_commissions')
set_subscriber_credit_limits = Bool('set_subscriber_credit_limits')
admin_terminate_accounts = Bool('admin_terminate_accounts')
admin_suspend_accounts = Bool('admin_suspend_accounts')
new_broker_credential_verification = Bool('new_broker_credential_verification')
broker_modify_closed_account_data = Bool('broker_modify_closed_account_data')
monthly_account_audits = Bool('monthly_account_audits')
weekly_account_audits = Bool('weekly_account_audits')
foreign_account_creation_allowed = Bool('foreign_account_creation_allowed')
inactive_foreign_account_locked = Bool('inactive_foreign_account_locked')
broker_print_rate_tables = Bool('broker_print_rate_tables')
broker_modify_new_subscription_accounts = Bool('broker_modify_new_subscription_accounts')
assign_custom_numbers = Bool('assign_custom_numbers')
highlight_overdue_invoices = Bool('highlight_overdue_invoices')
broker_delete_subscriber_invoices = Bool('broker_delete_subscriber_invoices')
overdue_invoice_configurable = Bool('overdue_invoice_configurable')
auto_generate_subscription_numbers = Bool('auto_generate_subscription_numbers')
invoices_flagged_overdue = Bool('invoices_flagged_overdue')
cleanup_scheduling = Bool('cleanup_scheduling')
overdue_invoices_deletion_allowed = Bool('overdue_invoices_deletion_allowed')
admin_view_closed_broker_accounts = Bool('admin_view_closed_broker_accounts')
reopen_closed_accounts = Bool('reopen_closed_accounts')
admin_reopen_closed_accounts = Bool('admin_reopen_closed_accounts')
payment_gateway_record_history = Bool('payment_gateway_record_history')
payment_history_backups = Bool('payment_history_backups')
prevent_invoice_table_backups = Bool('prevent_invoice_table_backups')
locked_foreign_accounts_closed = Bool('locked_foreign_accounts_closed')
inactive_foreign_accounts_closed = Bool('inactive_foreign_accounts_closed')
pbx_virtual_circuits_allowed = Bool('pbx_virtual_circuits_allowed')

constraints = [
    # After 3 failed login attempts, the system must not allow account access.
    Implies(failed_login_attempts >= 3, Not(account_access_allowed)),

    # When locking an account, the system must send a notification email.
    Implies(account_locked, notification_email_sent),

    # When locking an account, the system must send an SMS unless email was sent.
    Implies(And(account_locked, Not(notification_email_sent)), sms_sent),

    # The system must invoice brokers annually using CDR.
    invoice_annually,

    # The system must invoice brokers monthly using CDR.
    invoice_monthly,

    # The system must not allow invoice deletion.
    Not(invoice_deletion_allowed),

    # Brokers must be allowed to delete subscriber accounts and data.
    broker_delete_subscriber_allowed,

    # Invoices must only be stored in subscriber accounts.
    invoices_stored_in_subscriber_accounts,

    # PBX accounts must not have virtual circuits.
    Not(pbx_accounts_have_virtual_circuits),

    # The system must deactivate unpaid PBX virtual circuits.
    deactivate_unpaid_pbx_circuits,

    # Brokers must be allowed to access the rate table.
    broker_access_rate_table,

    # Only administrators must access and manage rate table permissions.
    admin_manage_rate_table_permissions,

    # The system must assign unique user IDs.
    unique_user_ids,

    # Users must be allowed to customize ID prefixes.
    customize_id_prefixes,

    # The system must send invoice/approval/payment notifications to brokers and subscribers.
    send_notifications,

    # The system must integrate with payment gateways.
    integrate_payment_gateways,

    # Payment history must be recorded and accessible.
    record_payment_history,

    # The system must log all user actions (deletions/modifications/accesses).
    log_user_actions,

    # The system must provide reporting/analytics for admins and brokers.
    reporting_analytics,

    # The system must support multi-currency invoicing/payments when required.
    multi_currency_support,

    # Monthly backups and recovery plans must be maintained.
    monthly_backups,

    # CRM/ERP integration must be available when required.
    crm_erp_integration,

    # Brokers must be allowed to bulk-generate invoices.
    bulk_generate_invoices,

    # Accounting software integration must be available.
    accounting_software_integration,

    # Brokers must have dashboards showing account status/earnings.
    broker_dashboards,

    # The system must calculate broker commissions per paid invoice.
    calculate_broker_commissions,

    # Brokers must be allowed to set subscriber credit limits.
    set_subscriber_credit_limits,

    # Admins must be allowed to terminate accounts for violations.
    admin_terminate_accounts,

    # Admins must be allowed to temporarily suspend accounts.
    admin_suspend_accounts,

    # New broker registrations must require credential verification.
    new_broker_credential_verification,

    # Brokers must not modify closed account data.
    Not(broker_modify_closed_account_data),

    # Monthly account audits must be performed.
    monthly_account_audits,

    # Weekly account audits must be performed.
    weekly_account_audits,

    # Foreign account creation must not be allowed.
    Not(foreign_account_creation_allowed),

    # Inactive foreign accounts (>3 months) must be locked.
    inactive_foreign_account_locked,

    # Brokers must be allowed to print rate tables.
    broker_print_rate_tables,

    # Brokers must be allowed to modify new subscription accounts.
    broker_modify_new_subscription_accounts,

    # Brokers must be allowed to assign custom numbers to new accounts.
    assign_custom_numbers,

    # Overdue invoices must be highlighted in dashboards.
    highlight_overdue_invoices,

    # Brokers must be allowed to delete subscriber accounts/invoices.
    broker_delete_subscriber_invoices,

    # Overdue invoice highlighting must be configurable.
    overdue_invoice_configurable,

    # The system must auto-generate unique subscription numbers.
    auto_generate_subscription_numbers,

    # Brokers must be allowed to modify closed account data.
    broker_modify_closed_account_data,

    # Invoices >60 days old must be flagged as overdue.
    invoices_flagged_overdue,

    # Flagged invoices must enter cleanup scheduling.
    Implies(invoices_flagged_overdue, cleanup_scheduling),

    # Overdue invoices must not be deleted.
    Not(overdue_invoices_deletion_allowed),

    # Admins must be allowed to view closed broker accounts.
    admin_view_closed_broker_accounts,

    # Users with closed-account view access must be allowed to reopen them.
    reopen_closed_accounts,

    # Admins must not be allowed to reopen closed accounts.
    Not(admin_reopen_closed_accounts),

    # Payment Gateway integration must record payment history.
    payment_gateway_record_history,

    # When payment history is recorded, monthly backups must occur.
    Implies(record_payment_history, monthly_backups),

    # Payment Gateway integration must prevent monthly invoice table backups.
    prevent_invoice_table_backups,

    # Inactive foreign accounts must be locked.
    inactive_foreign_account_locked,

    # Locked foreign accounts must be closed after 30 days.
    locked_foreign_accounts_closed,

    # Inactive foreign accounts must not be closed.
    Not(inactive_foreign_accounts_closed),

    # The system must not allow PBX virtual circuits.
    Not(pbx_virtual_circuits_allowed),

    # The system must deactivate unpaid PBX circuits.
    deactivate_unpaid_pbx_circuits
]

s = Solver()
s.add(constraints)

In [7]:
s.check()

unsat

In [8]:
find_unsat_subsets(constraints)

The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...
UNSAT subset 1:
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 2:
Implies(failed_login_attempts >= 3,
        Not(account_access_allowed))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 3:
broker_modify_closed_account_data
Implies(account_locked, notification_email_sent)
Not(broker_modify_closed_account_data)
---
UNSAT subset 4:
Implies(And(account_locked, Not(notification_email_sent)),
        sms_sent)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 5:
broker_modify_closed_account_data
invoice_annually
Not(broker_modify_closed_account_data)
---
UNSAT subset 6:
invoice_monthly
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 7:
Not(invoice_deletion_allowed)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)


[{Not(broker_modify_closed_account_data), broker_modify_closed_account_data},
 {Implies(failed_login_attempts >= 3,
          Not(account_access_allowed)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, notification_email_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(And(account_locked, Not(notification_email_sent)),
          sms_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_annually},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_monthly},
 {Not(broker_modify_closed_account_data),
  Not(invoice_deletion_allowed),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_delete_subscriber_allowed,
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),


## Google Gemini 2.0 Flash

In [9]:
from z3 import *

s = Solver()

failed_login_attempts = Int('failed_login_attempts')
account_access = Bool('account_access')
account_locked = Bool('account_locked')
email_sent = Bool('email_sent')
sms_sent = Bool('sms_sent')
invoice_brokers_annually = Bool('invoice_brokers_annually')
invoice_brokers_monthly = Bool('invoice_brokers_monthly')
invoice_deletion_allowed = Bool('invoice_deletion_allowed')
broker_delete_subscriber_accounts = Bool('broker_delete_subscriber_accounts')
invoices_stored_subscriber_accounts = Bool('invoices_stored_subscriber_accounts')
pbx_virtual_circuits = Bool('pbx_virtual_circuits')
deactivate_unpaid_pbx_circuits = Bool('deactivate_unpaid_pbx_circuits')
broker_access_rate_table = Bool('broker_access_rate_table')
admin_access_manage_rate_table = Bool('admin_access_manage_rate_table')
unique_user_ids = Bool('unique_user_ids')
user_customize_id_prefixes = Bool('user_customize_id_prefixes')
send_notifications_brokers_subscribers = Bool('send_notifications_brokers_subscribers')
integrate_payment_gateways = Bool('integrate_payment_gateways')
payment_history_recorded_accessible = Bool('payment_history_recorded_accessible')
log_user_actions = Bool('log_user_actions')
reporting_analytics_admins_brokers = Bool('reporting_analytics_admins_brokers')
multi_currency_invoicing_payments = Bool('multi_currency_invoicing_payments')
monthly_backups_recovery = Bool('monthly_backups_recovery')
crm_erp_integration = Bool('crm_erp_integration')
bulk_generate_invoices = Bool('bulk_generate_invoices')
accounting_software_integration = Bool('accounting_software_integration')
broker_dashboards_account_status_earnings = Bool('broker_dashboards_account_status_earnings')
calculate_broker_commissions = Bool('calculate_broker_commissions')
broker_credit_limits = Bool('broker_credit_limits')
admin_terminate_accounts_violations = Bool('admin_terminate_accounts_violations')
admin_suspend_accounts = Bool('admin_suspend_accounts')
credential_verification = Bool('credential_verification')
broker_modify_closed_account_data = Bool('broker_modify_closed_account_data')
monthly_account_audits = Bool('monthly_account_audits')
weekly_account_audits = Bool('weekly_account_audits')
foreign_account_creation = Bool('foreign_account_creation')
inactive_foreign_accounts = Bool('inactive_foreign_accounts')
broker_print_rate_tables = Bool('broker_print_rate_tables')
broker_modify_new_subscription_accounts = Bool('broker_modify_new_subscription_accounts')
broker_assign_custom_numbers = Bool('broker_assign_custom_numbers')
overdue_invoices_highlighted = Bool('overdue_invoices_highlighted')
broker_delete_subscriber_invoices = Bool('broker_delete_subscriber_invoices')
overdue_highlighting_configurable = Bool('overdue_highlighting_configurable')
auto_generate_subscription_numbers = Bool('auto_generate_subscription_numbers')
invoices_over_60_days_overdue = Bool('invoices_over_60_days_overdue')
flagged_invoices_cleanup = Bool('flagged_invoices_cleanup')
overdue_invoices_deleted = Bool('overdue_invoices_deleted')
admin_view_closed_broker_accounts = Bool('admin_view_closed_broker_accounts')
user_reopen_closed_accounts = Bool('user_reopen_closed_accounts')
payment_history_recorded = Bool('payment_history_recorded')
monthly_invoice_table_backups = Bool('monthly_invoice_table_backups')
locked_foreign_accounts = Bool('locked_foreign_accounts')

constraints = [
    Implies(failed_login_attempts > 3, Not(account_access)),
    Implies(account_locked, email_sent),
    Implies(account_locked, Or(sms_sent, email_sent)),
    invoice_brokers_annually,
    invoice_brokers_monthly,
    Not(invoice_deletion_allowed),
    broker_delete_subscriber_accounts,
    invoices_stored_subscriber_accounts,
    Not(pbx_virtual_circuits),
    deactivate_unpaid_pbx_circuits,
    broker_access_rate_table,
    admin_access_manage_rate_table,
    unique_user_ids,
    user_customize_id_prefixes,
    send_notifications_brokers_subscribers,
    integrate_payment_gateways,
    payment_history_recorded_accessible,
    log_user_actions,
    reporting_analytics_admins_brokers,
    multi_currency_invoicing_payments,
    monthly_backups_recovery,
    crm_erp_integration,
    bulk_generate_invoices,
    accounting_software_integration,
    broker_dashboards_account_status_earnings,
    calculate_broker_commissions,
    broker_credit_limits,
    admin_terminate_accounts_violations,
    admin_suspend_accounts,
    credential_verification,
    Not(broker_modify_closed_account_data),
    monthly_account_audits,
    weekly_account_audits,
    Not(foreign_account_creation),
    Implies(inactive_foreign_accounts, account_locked),
    broker_print_rate_tables,
    broker_modify_new_subscription_accounts,
    broker_assign_custom_numbers,
    overdue_invoices_highlighted,
    broker_delete_subscriber_invoices,
    overdue_highlighting_configurable,
    auto_generate_subscription_numbers,
    broker_modify_closed_account_data,
    Implies(invoices_over_60_days_overdue, And(overdue_invoices_highlighted, invoices_over_60_days_overdue)),
    Implies(invoices_over_60_days_overdue, flagged_invoices_cleanup),
    Not(overdue_invoices_deleted),
    admin_view_closed_broker_accounts,
    Implies(admin_view_closed_broker_accounts, user_reopen_closed_accounts),
    Not(And(admin_view_closed_broker_accounts, user_reopen_closed_accounts)),
    Implies(integrate_payment_gateways, payment_history_recorded),
    Implies(payment_history_recorded, monthly_backups_recovery),
    Implies(integrate_payment_gateways, Not(monthly_invoice_table_backups)),
    Implies(inactive_foreign_accounts, account_locked),
    Implies(locked_foreign_accounts, account_locked),
    Implies(locked_foreign_accounts, And(locked_foreign_accounts, account_locked)),
    Not(And(inactive_foreign_accounts, locked_foreign_accounts)),
    Not(pbx_virtual_circuits),
    deactivate_unpaid_pbx_circuits,
]

s.add(constraints)

In [10]:
s.check()

unsat

In [11]:
find_unsat_subsets(constraints)

The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...
UNSAT subset 1:
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 2:
Implies(failed_login_attempts > 3, Not(account_access))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 3:
Implies(account_locked, email_sent)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 4:
Implies(account_locked, Or(sms_sent, email_sent))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 5:
invoice_brokers_annually
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 6:
Not(broker_modify_closed_account_data)
broker_modify_closed_account_data
invoice_brokers_monthly
---
UNSAT subset 7:
Not(invoice_deletion_allowed)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 8:
broker_delete_subscri

[{Not(broker_modify_closed_account_data), broker_modify_closed_account_data},
 {Implies(failed_login_attempts > 3, Not(account_access)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, email_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, Or(sms_sent, email_sent)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_brokers_annually},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_brokers_monthly},
 {Not(broker_modify_closed_account_data),
  Not(invoice_deletion_allowed),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_delete_subscriber_accounts,
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoi

# Dataset Library

Requirements:
```
The system must display last full backup status
The system must display last incremental backup status
The system must display last planned system reboot
The system must display last unplanned system reboot
The system must display last software upgrade
The system must display current software version
The system must display pending transactions
The system must display log file sizes
The system must display item record counts
The system must display bibliographic record counts
The system must display patron record counts
The system must display database size
The system must display active processes
The system must process transactions in real-time
The admin module must only work on Mac clients
The system must allow record sets for batch updates
The system must allow record sets to limit queries
The system must allow record deletion with review
The system must log deletion errors
The system must allow undoing deletions
Admins must see all user privileges
Admins must control all user privileges
The admin module must only work on macOS
The admin module must use non-relational database
The admin module must not work with screen readers
The admin module must not work with accessibility tools
The system must allow branch-specific indicators
The system must allow systemwide indicators
The system must allow creating loan rules
The system must allow modifying loan rules
Loan rules must control checkouts
Loan rules must calculate loan periods
Loan rules must determine renewals
The system must log data changes
The system must provide undo functionality
The system should provide revision control
Request rules must check patron type
Request rules must check number of holds
Request rules must check account balance
Request rules must check item type
Request rules must check item status
Request rules must check location code
Admins must create report templates
Staff must access report templates
Staff must run templates unchanged
Staff must modify templates
The system must support active SFTP
The system must support passive SFTP
The system must configure SFTP per vendor
The system must provide patron account interface
Staff must see all patron fields per privileges
Staff must see all item fields per privileges
Patrons must see limited fields
The system must warn about record changes
The admin module must support ≤10 locations
Request rules must specify override permissions
The system must secure patron data transfers
The system must provide workstation console
The console must show workstation names
The console must show IP addresses
The console must manage sessions
The console must terminate sessions
The admin module must provide dev environment
The admin module must provide training environment
The system must allow config migration
The system must provide suppression rules
Rules must control authority record visibility
Rules must control bib record visibility
Rules must control order record visibility
Rules must control item record visibility
The system must report current record numbers
The system must report number of records
The system must report deleted records
The system must report purged records
The system must schedule maintenance
The system must schedule reports
The system must schedule data exports
The admin module must be part of library system
The admin module must produce standard HTML
The system must do live incremental backups
The system must do live full backups
The system must support admin macros
The system must support user macros
The system must support shortcuts
Web admin must work in Chrome v10+
Web admin must work in Safari v5+
The admin module must run on Windows server
The system must provide log dashboard
Upgrades must include documentation
Docs must cover server updates
Docs must cover client updates
Docs must list new features
Docs must list changed features
Docs must list removed features
Admin processes must be centralized
Central admin must accept multi-location input
Central admin must serve multiple locations
The system must support SFTP
The system must support SSL
The system must support SSH
The system must work with VNC
The system must work with Remote Desktop
Staff accounts must use templates
Account creation must have granular privileges
Account modification must have granular privileges
Account deletion must have granular privileges
The system must assign config file permissions
The system must allow log review during operation
Docs must be version-specific
Docs must be web-based
Docs must be indexed
Docs must be organized by function
Docs must be searchable
The system must use SQL database
The admin module must use relational database
The admin module must be mobile-only
The system must provide server console
The console must have software shutdown
The console must have software startup
The console must have server shutdown
The console must have server restart
The system must support third-party backups
The system must control record number access
Admins must view others' scheduled jobs
Admins must manage others' scheduled jobs
The system must use security groups
The system must use role-based access
The system must provide only batch processing
The system must handle MARC imports/exports
Admins must control table/field access
The system must allow configurable alerts
The system must support Windows clients
The system must control record visibility
The system must integrate with library modules
The system must provide report permissions
The admin module must support accessibility tools
```

## ChatGPT 4

In [12]:
from z3 import *

s = Solver()
constraints = []

# Define variables
DisplayLastFullBackup = Bool('DisplayLastFullBackup')
DisplayLastIncrementalBackup = Bool('DisplayLastIncrementalBackup')
DisplayLastPlannedReboot = Bool('DisplayLastPlannedReboot')
DisplayLastUnplannedReboot = Bool('DisplayLastUnplannedReboot')
DisplayLastSoftwareUpgrade = Bool('DisplayLastSoftwareUpgrade')
DisplayCurrentSoftwareVersion = Bool('DisplayCurrentSoftwareVersion')
DisplayPendingTransactions = Bool('DisplayPendingTransactions')
DisplayLogFileSizes = Bool('DisplayLogFileSizes')
DisplayItemRecordCounts = Bool('DisplayItemRecordCounts')
DisplayBibliographicRecordCounts = Bool('DisplayBibliographicRecordCounts')
DisplayPatronRecordCounts = Bool('DisplayPatronRecordCounts')
DisplayDatabaseSize = Bool('DisplayDatabaseSize')
DisplayActiveProcesses = Bool('DisplayActiveProcesses')
ProcessTransactionsRealTime = Bool('ProcessTransactionsRealTime')
AdminModuleMacOnly = Bool('AdminModuleMacOnly')
AllowBatchUpdates = Bool('AllowBatchUpdates')
AllowLimitQueries = Bool('AllowLimitQueries')
AllowRecordDeletionWithReview = Bool('AllowRecordDeletionWithReview')
LogDeletionErrors = Bool('LogDeletionErrors')
AllowUndoDeletions = Bool('AllowUndoDeletions')
AdminsSeeAllPrivileges = Bool('AdminsSeeAllPrivileges')
AdminsControlAllPrivileges = Bool('AdminsControlAllPrivileges')
AdminModuleNonRelationalDB = Bool('AdminModuleNonRelationalDB')
AdminModuleNoScreenReaders = Bool('AdminModuleNoScreenReaders')
AdminModuleNoAccessibilityTools = Bool('AdminModuleNoAccessibilityTools')
AllowBranchIndicators = Bool('AllowBranchIndicators')
AllowSystemwideIndicators = Bool('AllowSystemwideIndicators')
AllowLoanRulesCreation = Bool('AllowLoanRulesCreation')
AllowLoanRulesModification = Bool('AllowLoanRulesModification')
LoanRulesControlCheckouts = Bool('LoanRulesControlCheckouts')
LoanRulesCalculatePeriods = Bool('LoanRulesCalculatePeriods')
LoanRulesDetermineRenewals = Bool('LoanRulesDetermineRenewals')
LogDataChanges = Bool('LogDataChanges')
ProvideUndoFunctionality = Bool('ProvideUndoFunctionality')
ProvideRevisionControl = Bool('ProvideRevisionControl')
CheckPatronType = Bool('CheckPatronType')
CheckNumberOfHolds = Bool('CheckNumberOfHolds')
CheckAccountBalance = Bool('CheckAccountBalance')
CheckItemType = Bool('CheckItemType')
CheckItemStatus = Bool('CheckItemStatus')
CheckLocationCode = Bool('CheckLocationCode')
AdminsCreateReportTemplates = Bool('AdminsCreateReportTemplates')
StaffAccessReportTemplates = Bool('StaffAccessReportTemplates')
StaffRunTemplatesUnchanged = Bool('StaffRunTemplatesUnchanged')
StaffModifyTemplates = Bool('StaffModifyTemplates')
SupportActiveSFTP = Bool('SupportActiveSFTP')
SupportPassiveSFTP = Bool('SupportPassiveSFTP')
ConfigureSFTPPerVendor = Bool('ConfigureSFTPPerVendor')
ProvidePatronAccountInterface = Bool('ProvidePatronAccountInterface')
StaffSeePatronFieldsPerPrivileges = Bool('StaffSeePatronFieldsPerPrivileges')
StaffSeeItemFieldsPerPrivileges = Bool('StaffSeeItemFieldsPerPrivileges')
PatronsSeeLimitedFields = Bool('PatronsSeeLimitedFields')
WarnRecordChanges = Bool('WarnRecordChanges')
AdminModuleSupportMax10Locations = Bool('AdminModuleSupportMax10Locations')
SpecifyOverridePermissions = Bool('SpecifyOverridePermissions')
SecurePatronDataTransfers = Bool('SecurePatronDataTransfers')
ProvideWorkstationConsole = Bool('ProvideWorkstationConsole')
ConsoleShowWorkstationNames = Bool('ConsoleShowWorkstationNames')
ConsoleShowIPAddresses = Bool('ConsoleShowIPAddresses')
ConsoleManageSessions = Bool('ConsoleManageSessions')
ConsoleTerminateSessions = Bool('ConsoleTerminateSessions')
ProvideDevEnvironment = Bool('ProvideDevEnvironment')
ProvideTrainingEnvironment = Bool('ProvideTrainingEnvironment')
AllowConfigMigration = Bool('AllowConfigMigration')
ProvideSuppressionRules = Bool('ProvideSuppressionRules')
ControlAuthorityRecordVisibility = Bool('ControlAuthorityRecordVisibility')
ControlBibRecordVisibility = Bool('ControlBibRecordVisibility')
ControlOrderRecordVisibility = Bool('ControlOrderRecordVisibility')
ControlItemRecordVisibility = Bool('ControlItemRecordVisibility')
ReportCurrentRecordNumbers = Bool('ReportCurrentRecordNumbers')
ReportNumberOfRecords = Bool('ReportNumberOfRecords')
ReportDeletedRecords = Bool('ReportDeletedRecords')
ReportPurgedRecords = Bool('ReportPurgedRecords')
ScheduleMaintenance = Bool('ScheduleMaintenance')
ScheduleReports = Bool('ScheduleReports')
ScheduleDataExports = Bool('ScheduleDataExports')
AdminModulePartOfLibrarySystem = Bool('AdminModulePartOfLibrarySystem')
ProduceStandardHTML = Bool('ProduceStandardHTML')
LiveIncrementalBackups = Bool('LiveIncrementalBackups')
LiveFullBackups = Bool('LiveFullBackups')
SupportAdminMacros = Bool('SupportAdminMacros')
SupportUserMacros = Bool('SupportUserMacros')
SupportShortcuts = Bool('SupportShortcuts')
WebAdminChrome10Plus = Bool('WebAdminChrome10Plus')
WebAdminSafari5Plus = Bool('WebAdminSafari5Plus')
AdminModuleWindowsServer = Bool('AdminModuleWindowsServer')
ProvideLogDashboard = Bool('ProvideLogDashboard')
UpgradesIncludeDocumentation = Bool('UpgradesIncludeDocumentation')
DocsCoverServerUpdates = Bool('DocsCoverServerUpdates')
DocsCoverClientUpdates = Bool('DocsCoverClientUpdates')
DocsListNewFeatures = Bool('DocsListNewFeatures')
DocsListChangedFeatures = Bool('DocsListChangedFeatures')
DocsListRemovedFeatures = Bool('DocsListRemovedFeatures')
CentralAdminAcceptMultiLocation = Bool('CentralAdminAcceptMultiLocation')
CentralAdminServeMultipleLocations = Bool('CentralAdminServeMultipleLocations')
SupportSFTP = Bool('SupportSFTP')
SupportSSL = Bool('SupportSSL')
SupportSSH = Bool('SupportSSH')
WorkWithVNC = Bool('WorkWithVNC')
WorkWithRemoteDesktop = Bool('WorkWithRemoteDesktop')
StaffAccountsUseTemplates = Bool('StaffAccountsUseTemplates')
AccountCreationGranularPrivileges = Bool('AccountCreationGranularPrivileges')
AccountModificationGranularPrivileges = Bool('AccountModificationGranularPrivileges')
AccountDeletionGranularPrivileges = Bool('AccountDeletionGranularPrivileges')
AssignConfigFilePermissions = Bool('AssignConfigFilePermissions')
AllowLogReviewDuringOperation = Bool('AllowLogReviewDuringOperation')
DocsVersionSpecific = Bool('DocsVersionSpecific')
DocsWebBased = Bool('DocsWebBased')
DocsIndexed = Bool('DocsIndexed')
DocsOrganizedByFunction = Bool('DocsOrganizedByFunction')
DocsSearchable = Bool('DocsSearchable')
UseSQLDatabase = Bool('UseSQLDatabase')
AdminModuleRelationalDB = Bool('AdminModuleRelationalDB')
AdminModuleMobileOnly = Bool('AdminModuleMobileOnly')
ProvideServerConsole = Bool('ProvideServerConsole')
ConsoleSoftwareShutdown = Bool('ConsoleSoftwareShutdown')
ConsoleSoftwareStartup = Bool('ConsoleSoftwareStartup')
ConsoleServerShutdown = Bool('ConsoleServerShutdown')
ConsoleServerRestart = Bool('ConsoleServerRestart')
SupportThirdPartyBackups = Bool('SupportThirdPartyBackups')
ControlRecordNumberAccess = Bool('ControlRecordNumberAccess')
AdminsViewScheduledJobs = Bool('AdminsViewScheduledJobs')
AdminsManageScheduledJobs = Bool('AdminsManageScheduledJobs')
UseSecurityGroups = Bool('UseSecurityGroups')
UseRoleBasedAccess = Bool('UseRoleBasedAccess')
ProvideOnlyBatchProcessing = Bool('ProvideOnlyBatchProcessing')
HandleMARCImportsExports = Bool('HandleMARCImportsExports')
AdminsControlTableFieldAccess = Bool('AdminsControlTableFieldAccess')

s.add(constraints)

In [13]:
s.check()

sat

In [14]:
find_unsat_subsets(constraints)

The full set of constraints is satisfiable.


## Deepseek R1

In [25]:
from z3 import *

# Declare sorts
Location = DeclareSort('Location')
Patron = DeclareSort('Patron')
Item = DeclareSort('Item')
BibRecord = DeclareSort('BibRecord')
Staff = DeclareSort('Staff')
Admin = DeclareSort('Admin')
User = DeclareSort('User')
Record = DeclareSort('Record')
Transaction = DeclareSort('Transaction')
Process = DeclareSort('Process')
LoanRule = DeclareSort('LoanRule')
RequestRule = DeclareSort('RequestRule')
ReportTemplate = DeclareSort('ReportTemplate')
Session = DeclareSort('Session')
Workstation = DeclareSort('Workstation')
Macro = DeclareSort('Macro')
Shortcut = DeclareSort('Shortcut')
Config = DeclareSort('Config')
Alert = DeclareSort('Alert')
Job = DeclareSort('Job')
SecurityGroup = DeclareSort('SecurityGroup')
Role = DeclareSort('Role')
Module = DeclareSort('Module')

# Declare variables for quantification
a = Const('a', Admin)
u = Const('u', User)
l = Const('l', Location)
lr = Const('lr', LoanRule)
rr = Const('rr', RequestRule)
s = Const('s', Staff)
p = Const('p', Patron)
i = Const('i', Item)
rt = Const('rt', ReportTemplate)
w = Const('w', Workstation)
se = Const('se', Session)
c = Const('c', Config)
br = Const('br', BibRecord)
r = Const('r', Record)
m = Const('m', Macro)
sh = Const('sh', Shortcut)
j = Const('j', Job)
sg = Const('sg', SecurityGroup)
ro = Const('ro', Role)
mod = Const('mod', Module)
al = Const('al', Alert)
h = Int('h')  # Number of holds
b = Int('b')  # Account balance

# Declare predicates (boolean variables)
display_last_full_backup_status = Bool('display_last_full_backup_status')
display_last_incremental_backup_status = Bool('display_last_incremental_backup_status')
display_last_planned_reboot = Bool('display_last_planned_reboot')
display_last_unplanned_reboot = Bool('display_last_unplanned_reboot')
display_last_software_upgrade = Bool('display_last_software_upgrade')
display_current_software_version = Bool('display_current_software_version')
display_pending_transactions = Bool('display_pending_transactions')
display_log_file_sizes = Bool('display_log_file_sizes')
display_item_record_counts = Bool('display_item_record_counts')
display_bib_record_counts = Bool('display_bib_record_counts')
display_patron_record_counts = Bool('display_patron_record_counts')
display_database_size = Bool('display_database_size')
display_active_processes = Bool('display_active_processes')
process_transactions_realtime = Bool('process_transactions_realtime')
admin_module_mac_only = Bool('admin_module_mac_only')
allow_record_sets_batch = Bool('allow_record_sets_batch')
allow_record_sets_queries = Bool('allow_record_sets_queries')
allow_deletion_with_review = Bool('allow_deletion_with_review')
log_deletion_errors = Bool('log_deletion_errors')
allow_undo_deletions = Bool('allow_undo_deletions')
admin_module_macos_only = Bool('admin_module_macos_only')
admin_module_nonrelational_db = Bool('admin_module_nonrelational_db')
admin_module_no_screen_readers = Bool('admin_module_no_screen_readers')
admin_module_no_accessibility_tools = Bool('admin_module_no_accessibility_tools')
allow_systemwide_indicators = Bool('allow_systemwide_indicators')
allow_create_loan_rules = Bool('allow_create_loan_rules')
log_data_changes = Bool('log_data_changes')
provide_undo_functionality = Bool('provide_undo_functionality')
provide_revision_control = Bool('provide_revision_control')
support_active_sftp = Bool('support_active_sftp')
support_passive_sftp = Bool('support_passive_sftp')
configure_sftp_per_vendor = Bool('configure_sftp_per_vendor')
provide_patron_account_interface = Bool('provide_patron_account_interface')
warn_record_changes = Bool('warn_record_changes')
admin_support_10_locations = Bool('admin_support_10_locations')
secure_patron_data_transfers = Bool('secure_patron_data_transfers')
provide_workstation_console = Bool('provide_workstation_console')
admin_provide_dev_env = Bool('admin_provide_dev_env')
admin_provide_training_env = Bool('admin_provide_training_env')
provide_suppression_rules = Bool('provide_suppression_rules')
report_current_record_numbers = Bool('report_current_record_numbers')
report_number_of_records = Bool('report_number_of_records')
report_deleted_records = Bool('report_deleted_records')
report_purged_records = Bool('report_purged_records')
schedule_maintenance = Bool('schedule_maintenance')
schedule_reports = Bool('schedule_reports')
schedule_data_exports = Bool('schedule_data_exports')
admin_produce_standard_html = Bool('admin_produce_standard_html')
do_live_incremental_backups = Bool('do_live_incremental_backups')
do_live_full_backups = Bool('do_live_full_backups')
web_admin_chrome_v10 = Bool('web_admin_chrome_v10')
web_admin_safari_v5 = Bool('web_admin_safari_v5')
admin_run_windows_server = Bool('admin_run_windows_server')
provide_log_dashboard = Bool('provide_log_dashboard')
upgrades_include_docs = Bool('upgrades_include_docs')
docs_cover_server_updates = Bool('docs_cover_server_updates')
docs_cover_client_updates = Bool('docs_cover_client_updates')
docs_list_new_features = Bool('docs_list_new_features')
docs_list_changed_features = Bool('docs_list_changed_features')
docs_list_removed_features = Bool('docs_list_removed_features')
admin_processes_centralized = Bool('admin_processes_centralized')
support_sftp = Bool('support_sftp')
support_ssl = Bool('support_ssl')
support_ssh = Bool('support_ssh')
work_with_vnc = Bool('work_with_vnc')
work_with_remote_desktop = Bool('work_with_remote_desktop')
allow_log_review_during_operation = Bool('allow_log_review_during_operation')
docs_version_specific = Bool('docs_version_specific')
docs_web_based = Bool('docs_web_based')
docs_indexed = Bool('docs_indexed')
docs_organized_by_function = Bool('docs_organized_by_function')
docs_searchable = Bool('docs_searchable')
use_sql_database = Bool('use_sql_database')
admin_use_relational_db = Bool('admin_use_relational_db')
admin_mobile_only = Bool('admin_mobile_only')
provide_server_console = Bool('provide_server_console')
console_software_shutdown = Bool('console_software_shutdown')
console_software_startup = Bool('console_software_startup')
console_server_shutdown = Bool('console_server_shutdown')
console_server_restart = Bool('console_server_restart')
support_third_party_backups = Bool('support_third_party_backups')
provide_only_batch_processing = Bool('provide_only_batch_processing')
handle_marc_imports_exports = Bool('handle_marc_imports_exports')
support_windows_clients = Bool('support_windows_clients')
admin_support_accessibility_tools = Bool('admin_support_accessibility_tools')

# Declare functions
admin_see_all_privileges = Function('admin_see_all_privileges', Admin, User, BoolSort())
admin_control_privileges = Function('admin_control_privileges', Admin, User, BoolSort())
allow_branch_indicators = Function('allow_branch_indicators', Location, BoolSort())
allow_modify_loan_rules = Function('allow_modify_loan_rules', LoanRule, BoolSort())
loan_rules_control_checkouts = Function('loan_rules_control_checkouts', LoanRule, BoolSort())
loan_rules_calculate_periods = Function('loan_rules_calculate_periods', LoanRule, BoolSort())
loan_rules_determine_renewals = Function('loan_rules_determine_renewals', LoanRule, BoolSort())
request_rules_check_patron_type = Function('request_rules_check_patron_type', RequestRule, Patron, BoolSort())
request_rules_check_holds = Function('request_rules_check_holds', RequestRule, IntSort(), BoolSort())
request_rules_check_balance = Function('request_rules_check_balance', RequestRule, IntSort(), BoolSort())
request_rules_check_item_type = Function('request_rules_check_item_type', RequestRule, Item, BoolSort())
request_rules_check_item_status = Function('request_rules_check_item_status', RequestRule, Item, BoolSort())
request_rules_check_location = Function('request_rules_check_location', RequestRule, Location, BoolSort())
admin_create_report_templates = Function('admin_create_report_templates', Admin, ReportTemplate, BoolSort())
staff_access_templates = Function('staff_access_templates', Staff, ReportTemplate, BoolSort())
staff_run_templates = Function('staff_run_templates', Staff, ReportTemplate, BoolSort())
staff_modify_templates = Function('staff_modify_templates', Staff, ReportTemplate, BoolSort())
staff_see_patron_fields = Function('staff_see_patron_fields', Staff, Patron, BoolSort())
staff_see_item_fields = Function('staff_see_item_fields', Staff, Item, BoolSort())
patrons_see_limited_fields = Function('patrons_see_limited_fields', Patron, BoolSort())
request_rules_override_permissions = Function('request_rules_override_permissions', RequestRule, BoolSort())
console_show_workstation_names = Function('console_show_workstation_names', Workstation, BoolSort())
console_show_ip_addresses = Function('console_show_ip_addresses', Workstation, BoolSort())
console_manage_sessions = Function('console_manage_sessions', Session, BoolSort())
console_terminate_sessions = Function('console_terminate_sessions', Session, BoolSort())
allow_config_migration = Function('allow_config_migration', Config, BoolSort())
rules_control_authority_visibility = Function('rules_control_authority_visibility', Record, BoolSort())
rules_control_bib_visibility = Function('rules_control_bib_visibility', BibRecord, BoolSort())
rules_control_order_visibility = Function('rules_control_order_visibility', Record, BoolSort())
rules_control_item_visibility = Function('rules_control_item_visibility', Item, BoolSort())
admin_part_of_library_system = Function('admin_part_of_library_system', Module, BoolSort())
support_admin_macros = Function('support_admin_macros', Macro, BoolSort())
support_user_macros = Function('support_user_macros', User, Macro, BoolSort())
support_shortcuts = Function('support_shortcuts', Shortcut, BoolSort())
central_admin_multi_location_input = Function('central_admin_multi_location_input', Location, BoolSort())
central_admin_serve_multi_locations = Function('central_admin_serve_multi_locations', Location, BoolSort())
staff_accounts_use_templates = Function('staff_accounts_use_templates', Staff, BoolSort())
account_creation_granular_privileges = Function('account_creation_granular_privileges', User, BoolSort())
account_modification_granular_privileges = Function('account_modification_granular_privileges', User, BoolSort())
account_deletion_granular_privileges = Function('account_deletion_granular_privileges', User, BoolSort())
assign_config_permissions = Function('assign_config_permissions', Config, BoolSort())
control_record_number_access = Function('control_record_number_access', Record, BoolSort())
admin_view_others_jobs = Function('admin_view_others_jobs', Admin, Job, BoolSort())
admin_manage_others_jobs = Function('admin_manage_others_jobs', Admin, Job, BoolSort())
use_security_groups = Function('use_security_groups', SecurityGroup, BoolSort())
use_role_based_access = Function('use_role_based_access', Role, BoolSort())
admin_control_table_field_access = Function('admin_control_table_field_access', Admin, Record, BoolSort())
allow_configurable_alerts = Function('allow_configurable_alerts', Alert, BoolSort())
control_record_visibility = Function('control_record_visibility', Record, BoolSort())
integrate_with_library_modules = Function('integrate_with_library_modules', Module, BoolSort())
provide_report_permissions = Function('provide_report_permissions', ReportTemplate, BoolSort())

# Constraints
constraints = [
    display_last_full_backup_status,
    display_last_incremental_backup_status,
    display_last_planned_reboot,
    display_last_unplanned_reboot,
    display_last_software_upgrade,
    display_current_software_version,
    display_pending_transactions,
    display_log_file_sizes,
    display_item_record_counts,
    display_bib_record_counts,
    display_patron_record_counts,
    display_database_size,
    display_active_processes,
    process_transactions_realtime,
    admin_module_mac_only,
    allow_record_sets_batch,
    allow_record_sets_queries,
    allow_deletion_with_review,
    log_deletion_errors,
    allow_undo_deletions,
    ForAll([a, u], admin_see_all_privileges(a, u)),
    ForAll([a, u], admin_control_privileges(a, u)),
    admin_module_macos_only,
    admin_module_nonrelational_db,
    admin_module_no_screen_readers,
    admin_module_no_accessibility_tools,
    ForAll([l], allow_branch_indicators(l)),
    allow_systemwide_indicators,
    allow_create_loan_rules,
    ForAll([lr], allow_modify_loan_rules(lr)),
    ForAll([lr], loan_rules_control_checkouts(lr)),
    ForAll([lr], loan_rules_calculate_periods(lr)),
    ForAll([lr], loan_rules_determine_renewals(lr)),
    log_data_changes,
    provide_undo_functionality,
    provide_revision_control,
    ForAll([rr, p], request_rules_check_patron_type(rr, p)),
    ForAll([rr, h], request_rules_check_holds(rr, h)),
    ForAll([rr, b], request_rules_check_balance(rr, b)),
    ForAll([rr, i], request_rules_check_item_type(rr, i)),
    ForAll([rr, i], request_rules_check_item_status(rr, i)),
    ForAll([rr, l], request_rules_check_location(rr, l)),
    ForAll([a, rt], admin_create_report_templates(a, rt)),
    ForAll([s, rt], staff_access_templates(s, rt)),
    ForAll([s, rt], staff_run_templates(s, rt)),
    ForAll([s, rt], staff_modify_templates(s, rt)),
    support_active_sftp,
    support_passive_sftp,
    configure_sftp_per_vendor,
    provide_patron_account_interface,
    ForAll([s, p], staff_see_patron_fields(s, p)),
    ForAll([s, i], staff_see_item_fields(s, i)),
    ForAll([p], patrons_see_limited_fields(p)),
    warn_record_changes,
    admin_support_10_locations,
    ForAll([rr], request_rules_override_permissions(rr)),
    secure_patron_data_transfers,
    provide_workstation_console,
    ForAll([w], console_show_workstation_names(w)),
    ForAll([w], console_show_ip_addresses(w)),
    ForAll([se], console_manage_sessions(se)),
    ForAll([se], console_terminate_sessions(se)),
    admin_provide_dev_env,
    admin_provide_training_env,
    ForAll([c], allow_config_migration(c)),
    provide_suppression_rules,
    ForAll([r], rules_control_authority_visibility(r)),
    ForAll([br], rules_control_bib_visibility(br)),
    ForAll([r], rules_control_order_visibility(r)),
    ForAll([i], rules_control_item_visibility(i)),
    report_current_record_numbers,
    report_number_of_records,
    report_deleted_records,
    report_purged_records,
    schedule_maintenance,
    schedule_reports,
    schedule_data_exports,
    ForAll([mod], admin_part_of_library_system(mod)),
    admin_produce_standard_html,
    do_live_incremental_backups,
    do_live_full_backups,
    ForAll([m], support_admin_macros(m)),
    ForAll([u, m], support_user_macros(u, m)),
    ForAll([sh], support_shortcuts(sh)),
    web_admin_chrome_v10,
    web_admin_safari_v5,
    admin_run_windows_server,
    provide_log_dashboard,
    upgrades_include_docs,
    docs_cover_server_updates,
    docs_cover_client_updates,
    docs_list_new_features,
    docs_list_changed_features,
    docs_list_removed_features,
    admin_processes_centralized,
    ForAll([l], central_admin_multi_location_input(l)),
    ForAll([l], central_admin_serve_multi_locations(l)),
    support_sftp,
    support_ssl,
    support_ssh,
    work_with_vnc,
    work_with_remote_desktop,
    ForAll([s], staff_accounts_use_templates(s)),
    ForAll([u], account_creation_granular_privileges(u)),
    ForAll([u], account_modification_granular_privileges(u)),
    ForAll([u], account_deletion_granular_privileges(u)),
    ForAll([c], assign_config_permissions(c)),
    allow_log_review_during_operation,
    docs_version_specific,
    docs_web_based,
    docs_indexed,
    docs_organized_by_function,
    docs_searchable,
    use_sql_database,
    admin_use_relational_db,
    admin_mobile_only,
    provide_server_console,
    console_software_shutdown,
    console_software_startup,
    console_server_shutdown,
    console_server_restart,
    support_third_party_backups,
    ForAll([r], control_record_number_access(r)),
    ForAll([a, j], admin_view_others_jobs(a, j)),
    ForAll([a, j], admin_manage_others_jobs(a, j)),
    ForAll([sg], use_security_groups(sg)),
    ForAll([ro], use_role_based_access(ro)),
    provide_only_batch_processing,
    handle_marc_imports_exports,
    ForAll([a, r], admin_control_table_field_access(a, r)),
    ForAll([al], allow_configurable_alerts(al)),
    support_windows_clients,
    ForAll([r], control_record_visibility(r)),
    ForAll([mod], integrate_with_library_modules(mod)),
    ForAll([rt], provide_report_permissions(rt)),
    admin_support_accessibility_tools
]

s = Solver()
s.add(constraints)

In [26]:
s.check()

sat

In [27]:
find_unsat_subsets(constraints)

The full set of constraints is satisfiable.


## Google Gemini 2.0 Flash

In [15]:
from z3 import *

s = Solver()

display_last_full_backup_status = Bool('display_last_full_backup_status')
display_last_incremental_backup_status = Bool('display_last_incremental_backup_status')
display_last_planned_system_reboot = Bool('display_last_planned_system_reboot')
display_last_unplanned_system_reboot = Bool('display_last_unplanned_system_reboot')
display_last_software_upgrade = Bool('display_last_software_upgrade')
display_current_software_version = Bool('display_current_software_version')
display_pending_transactions = Bool('display_pending_transactions')
display_log_file_sizes = Bool('display_log_file_sizes')
display_item_record_counts = Bool('display_item_record_counts')
display_bibliographic_record_counts = Bool('display_bibliographic_record_counts')
display_patron_record_counts = Bool('display_patron_record_counts')
display_database_size = Bool('display_database_size')
display_active_processes = Bool('display_active_processes')
process_transactions_real_time = Bool('process_transactions_real_time')
admin_module_mac_clients = Bool('admin_module_mac_clients')
allow_record_sets_batch_updates = Bool('allow_record_sets_batch_updates')
allow_record_sets_limit_queries = Bool('allow_record_sets_limit_queries')
allow_record_deletion_review = Bool('allow_record_deletion_review')
log_deletion_errors = Bool('log_deletion_errors')
allow_undoing_deletions = Bool('allow_undoing_deletions')
admins_see_user_privileges = Bool('admins_see_user_privileges')
admins_control_user_privileges = Bool('admins_control_user_privileges')
admin_module_macos = Bool('admin_module_macos')
admin_module_non_relational_database = Bool('admin_module_non_relational_database')
admin_module_no_screen_readers = Bool('admin_module_no_screen_readers')
admin_module_no_accessibility_tools = Bool('admin_module_no_accessibility_tools')
allow_branch_specific_indicators = Bool('allow_branch_specific_indicators')
allow_systemwide_indicators = Bool('allow_systemwide_indicators')
allow_create_loan_rules = Bool('allow_create_loan_rules')
allow_modify_loan_rules = Bool('allow_modify_loan_rules')
loan_rules_control_checkouts = Bool('loan_rules_control_checkouts')
loan_rules_calculate_loan_periods = Bool('loan_rules_calculate_loan_periods')
loan_rules_determine_renewals = Bool('loan_rules_determine_renewals')
log_data_changes = Bool('log_data_changes')
provide_undo_functionality = Bool('provide_undo_functionality')
provide_revision_control = Bool('provide_revision_control')
request_rules_check_patron_type = Bool('request_rules_check_patron_type')
request_rules_check_number_holds = Bool('request_rules_check_number_holds')
request_rules_check_account_balance = Bool('request_rules_check_account_balance')
request_rules_check_item_type = Bool('request_rules_check_item_type')
request_rules_check_item_status = Bool('request_rules_check_item_status')
request_rules_check_location_code = Bool('request_rules_check_location_code')
admins_create_report_templates = Bool('admins_create_report_templates')
staff_access_report_templates = Bool('staff_access_report_templates')
staff_run_templates_unchanged = Bool('staff_run_templates_unchanged')
staff_modify_templates = Bool('staff_modify_templates')
support_active_SFTP = Bool('support_active_SFTP')
support_passive_SFTP = Bool('support_passive_SFTP')
configure_SFTP_per_vendor = Bool('configure_SFTP_per_vendor')
provide_patron_account_interface = Bool('provide_patron_account_interface')
staff_see_all_patron_fields = Bool('staff_see_all_patron_fields')
staff_see_all_item_fields = Bool('staff_see_all_item_fields')
patrons_see_limited_fields = Bool('patrons_see_limited_fields')
warn_record_changes = Bool('warn_record_changes')
admin_module_less_than_10_locations = Bool('admin_module_less_than_10_locations')
request_rules_specify_override_permissions = Bool('request_rules_specify_override_permissions')
secure_patron_data_transfers = Bool('secure_patron_data_transfers')
provide_workstation_console = Bool('provide_workstation_console')
console_show_workstation_names = Bool('console_show_workstation_names')
console_show_IP_addresses = Bool('console_show_IP_addresses')
console_manage_sessions = Bool('console_manage_sessions')
console_terminate_sessions = Bool('console_terminate_sessions')
admin_module_dev_environment = Bool('admin_module_dev_environment')
admin_module_training_environment = Bool('admin_module_training_environment')
allow_config_migration = Bool('allow_config_migration')
provide_suppression_rules = Bool('provide_suppression_rules')
rules_control_authority_record_visibility = Bool('rules_control_authority_record_visibility')
rules_control_bib_record_visibility = Bool('rules_control_bib_record_visibility')
rules_control_order_record_visibility = Bool('rules_control_order_record_visibility')
rules_control_item_record_visibility = Bool('rules_control_item_record_visibility')
report_current_record_numbers = Bool('report_current_record_numbers')
report_number_records = Bool('report_number_records')
report_deleted_records = Bool('report_deleted_records')
report_purged_records = Bool('report_purged_records')
schedule_maintenance = Bool('schedule_maintenance')
schedule_reports = Bool('schedule_reports')
schedule_data_exports = Bool('schedule_data_exports')
admin_module_part_library_system = Bool('admin_module_part_library_system')
admin_module_produce_standard_HTML = Bool('admin_module_produce_standard_HTML')
do_live_incremental_backups = Bool('do_live_incremental_backups')
do_live_full_backups = Bool('do_live_full_backups')
support_admin_macros = Bool('support_admin_macros')
support_user_macros = Bool('support_user_macros')
support_shortcuts = Bool('support_shortcuts')
web_admin_chrome_v10_plus = Bool('web_admin_chrome_v10_plus')
web_admin_safari_v5_plus = Bool('web_admin_safari_v5_plus')
admin_module_windows_server = Bool('admin_module_windows_server')
provide_log_dashboard = Bool('provide_log_dashboard')
upgrades_include_documentation = Bool('upgrades_include_documentation')
docs_cover_server_updates = Bool('docs_cover_server_updates')
docs_cover_client_updates = Bool('docs_cover_client_updates')
docs_list_new_features = Bool('docs_list_new_features')
docs_list_changed_features = Bool('docs_list_changed_features')
docs_list_removed_features = Bool('docs_list_removed_features')
admin_processes_centralized = Bool('admin_processes_centralized')
central_admin_multi_location_input = Bool('central_admin_multi_location_input')
central_admin_serve_multiple_locations = Bool('central_admin_serve_multiple_locations')
support_SFTP = Bool('support_SFTP')
support_SSL = Bool('support_SSL')
support_SSH = Bool('support_SSH')
work_with_VNC = Bool('work_with_VNC')
work_with_Remote_Desktop = Bool('work_with_Remote_Desktop')
staff_accounts_use_templates = Bool('staff_accounts_use_templates')
account_creation_granular_privileges = Bool('account_creation_granular_privileges')
account_modification_granular_privileges = Bool('account_modification_granular_privileges')
account_deletion_granular_privileges = Bool('account_deletion_granular_privileges')
assign_config_file_permissions = Bool('assign_config_file_permissions')
allow_log_review_during_operation = Bool('allow_log_review_during_operation')
docs_version_specific = Bool('docs_version_specific')
docs_web_based = Bool('docs_web_based')
docs_indexed = Bool('docs_indexed')
docs_organized_by_function = Bool('docs_organized_by_function')
docs_searchable = Bool('docs_searchable')
use_SQL_database = Bool('use_SQL_database')
admin_module_relational_database = Bool('admin_module_relational_database')
admin_module_mobile_only = Bool('admin_module_mobile_only')
provide_server_console = Bool('provide_server_console')
console_software_shutdown = Bool('console_software_shutdown')
console_software_startup = Bool('console_software_startup')
console_server_shutdown = Bool('console_server_shutdown')
console_server_restart = Bool('console_server_restart')
support_third_party_backups = Bool('support_third_party_backups')
control_record_number_access = Bool('control_record_number_access')
admins_view_scheduled_jobs = Bool('admins_view_scheduled_jobs')
admins_manage_scheduled_jobs = Bool('admins_manage_scheduled_jobs')
use_security_groups = Bool('use_security_groups')
use_role_based_access = Bool('use_role_based_access')
provide_batch_processing_only = Bool('provide_batch_processing_only')
handle_MARC_imports_exports = Bool('handle_MARC_imports_exports')
admins_control_table_field_access = Bool('admins_control_table_field_access')
allow_configurable_alerts = Bool('allow_configurable_alerts')
support_windows_clients = Bool('support_windows_clients')
control_record_visibility = Bool('control_record_visibility')
integrate_library_modules = Bool('integrate_library_modules')
provide_report_permissions = Bool('provide_report_permissions')
admin_module_accessibility_tools = Bool('admin_module_accessibility_tools')

constraints = [
    display_last_full_backup_status,
    display_last_incremental_backup_status,
    display_last_planned_system_reboot,
    display_last_unplanned_system_reboot,
    display_last_software_upgrade,
    display_current_software_version,
    display_pending_transactions,
    display_log_file_sizes,
    display_item_record_counts,
    display_bibliographic_record_counts,
    display_patron_record_counts,
    display_database_size,
    display_active_processes,
    process_transactions_real_time,
    admin_module_mac_clients,
    allow_record_sets_batch_updates,
    allow_record_sets_limit_queries,
    allow_record_deletion_review,
    log_deletion_errors,
    allow_undoing_deletions,
    admins_see_user_privileges,
    admins_control_user_privileges,
    admin_module_macos,
    admin_module_non_relational_database,
    Not(admin_module_no_screen_readers),
    Not(admin_module_no_accessibility_tools),
    allow_branch_specific_indicators,
    allow_systemwide_indicators,
    allow_create_loan_rules,
    allow_modify_loan_rules,
    loan_rules_control_checkouts,
    loan_rules_calculate_loan_periods,
    loan_rules_determine_renewals,
    log_data_changes,
    provide_undo_functionality,
    provide_revision_control,
    request_rules_check_patron_type,
    request_rules_check_number_holds,
    request_rules_check_account_balance,
    request_rules_check_item_type,
    request_rules_check_item_status,
    request_rules_check_location_code,
    admins_create_report_templates,
    staff_access_report_templates,
    staff_run_templates_unchanged,
    staff_modify_templates,
    support_active_SFTP,
    support_passive_SFTP,
    configure_SFTP_per_vendor,
    provide_patron_account_interface,
    staff_see_all_patron_fields,
    staff_see_all_item_fields,
    patrons_see_limited_fields,
    warn_record_changes,
    admin_module_less_than_10_locations,
    request_rules_specify_override_permissions,
    secure_patron_data_transfers,
    provide_workstation_console,
    console_show_workstation_names,
    console_show_IP_addresses,
    console_manage_sessions,
    console_terminate_sessions,
    admin_module_dev_environment,
    admin_module_training_environment,
    allow_config_migration,
    provide_suppression_rules,
    rules_control_authority_record_visibility,
    rules_control_bib_record_visibility,
    rules_control_order_record_visibility,
    rules_control_item_record_visibility,
    report_current_record_numbers,
    report_number_records,
    report_deleted_records,
    report_purged_records,
    schedule_maintenance,
    schedule_reports,
    schedule_data_exports,
    admin_module_part_library_system,
    admin_module_produce_standard_HTML,
    do_live_incremental_backups,
    do_live_full_backups,
    support_admin_macros,
    support_user_macros,
    support_shortcuts,
    web_admin_chrome_v10_plus,
    web_admin_safari_v5_plus,
    admin_module_windows_server,
    provide_log_dashboard,
    upgrades_include_documentation,
    docs_cover_server_updates,
    docs_cover_client_updates,
    docs_list_new_features,
    docs_list_changed_features,
    docs_list_removed_features,
    admin_processes_centralized,
    central_admin_multi_location_input,
    central_admin_serve_multiple_locations,
    support_SFTP,
    support_SSL,
    support_SSH,
    work_with_VNC,
    work_with_Remote_Desktop,
    staff_accounts_use_templates,
    account_creation_granular_privileges,
    account_modification_granular_privileges,
    account_deletion_granular_privileges,
    assign_config_file_permissions,
    allow_log_review_during_operation,
    docs_version_specific,
    docs_web_based,
    docs_indexed,
    docs_organized_by_function,
    docs_searchable,
    use_SQL_database,
    admin_module_relational_database,
    Not(admin_module_mobile_only),
    provide_server_console,
    console_software_shutdown,
    console_software_startup,
    console_server_shutdown,
    console_server_restart,
    support_third_party_backups,
    control_record_number_access,
    admins_view_scheduled_jobs,
    admins_manage_scheduled_jobs,
    use_security_groups,
    use_role_based_access,
    provide_batch_processing_only,
    handle_MARC_imports_exports,
    admins_control_table_field_access,
    allow_configurable_alerts,
    support_windows_clients,
    control_record_visibility,
    integrate_library_modules,
    provide_report_permissions,
    admin_module_accessibility_tools
]

s.add(constraints)

In [16]:
s.check()

sat

In [17]:
find_unsat_subsets(constraints)

The full set of constraints is satisfiable.
